In [1]:
import os, time, re, random, glob, json, jieba, copy
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from transformers import (
    AutoModel,
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    AutoModelForMultipleChoice,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    default_data_collator,
    TextGenerationPipeline
)

In [2]:
device="cuda:0" if torch.cuda.is_available() else "cpu"
from sys import platform
if platform == "linux" or platform == "linux2":
    # linux
    root = "/mnt/private-pa002-vol726121-prd/Data"
#         root = "/root/autodl-tmp/Data"
elif platform == "darwin":
    # OS X
    root = "/Users/zeyesun/Documents/Data"
elif platform == "win32":
    # Windows...
    root = "D:\\Data"

In [3]:
CLEAN_TEXT_PATTERN = re.compile(r"[\r\n]")

def clean_text(text):
    return CLEAN_TEXT_PATTERN.sub("", text)

In [4]:
# model_name_or_path = os.path.join(root, "models", "pangu-350M")
# model_name_or_path = os.path.join(root, "models", "pangu-2.6B")
# model_name_or_path = os.path.join(root, "models", "pangu-13B")
model_name_or_path = os.path.join(root, "models", "glm-335M-chinese")
# model_name_or_path = os.path.join(root, "models", "glm-10B-chinese")
# model_name_or_path = os.path.join(root, "models", "chatglm-6B")
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_cache=False, trust_remote_code=True)

# import sentencepiece
# model_file = os.path.join(root, "models", "pangu-350M", "vocab.model")
# sp = sentencepiece.SentencePieceProcessor()
# sp.Load(model_file=model_file)
# for i in range(10):
#     print(sp.id_to_piece(i))

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [38]:
max_length = 512
for prompt, target in zip(prompts_processed, labels):
    t1 = tokenizer.tokenize(prompt)
    t2 = tokenizer.tokenize(target)
    if len(t1) + len(t2) > max_length:
        print(f"{prompt}\n{target}\n\n")

今天晚上我在睡觉.........他想要做那些事..我就大大声骂他"不要吵我睡觉"!!!!!...他就跑出去了...还不接我电话<sep>回答：
你机器里同时有国服和台服是不行的.那样会有冲突的.无法正常运行的.以下办法用下吧.台服客户端下载：  (1)打开CWOW文件夹，找到realmlist.wtf文件，用记事本打开它，先清空里面的内容，再添加“set realmlist  ”（不包括双引号），保存后退出，这样，在每次要进去台服的时候，只需要用WoW.exe来启动，如果是进去CWOW的话，就用Launcher.exe来启动，不会造成冲突。另外，用WoW.exe启动后，登陆的界面是CWOW的界面，这是因为使用的是CWOW的客户端的原因，不用怀疑，直接输入ID密码登陆后进入的就是台服。 ---------------------- (2)如果你已经有CWOW的客户端了，再想装TWOW的时候需要修改注册表。 修改方法： 1、　点击　开始 2、　点击　运行 3、　在运行框里输入　regedit4、　在弹出的注册表框里点击　编辑 5、　点击查找 6、　在查找框里输入　Blizzard Entertainment 7、　当左边框里搜索到　Blizzard Entertainment　的时候，在左边整个删除，一直搜索到出现对话框里说　注册表搜索完毕　为止 [安装] 然后是安装，直接解压，安装就OK了，安装后貌似是1.11.0的版本(视你下载的版本为定)。 --------------------- 补丁方面,直接通过程序下载,速度很快,一般能达到80K/S以上的速度(1M ADSL).---------------------------【注册账号】此处资料为转载，仅供参考------  直接申请体验CDK,然后开始录入个人资料. 召集令编号: RF070112000032 RF070112000033 RF070112000034 RF070112000035 RF070112000036 注：注册帐号时填写"召集令编号"可获得（14天）30小时体验优惠。 非必选项 录入个人资料这里,请注意一下,请尽量填写个人准确资料,如地区选(其它),以及真实身份证号码,真实住址,还有真实的email地址.(^^我选的住在钓鱼岛...不过地址栏下面的都写的真实的) 范例：南海诸岛钓鱼台列

 01 切　沃(06) VS 巴　里(13) 30切沃最近的表现可以用渐入佳境来形容，先是连败两场，然后是五场不败，球队固定的主力阵型和清晰的打法套路显示出了教练的能力。巴里的表现其实也不差，联赛头轮打平国米，让人惊叹，接着大胜亚特兰大更是出乎意料。虽然说最近只获得了4平1负的成绩，但是其中也逼平了AC米兰和巴勒莫，还是比较有实力的。赔率方面，立博和威廉都支持分胜负，澳盘的盘口就相当诡异，先是主队让平半低水，然后直接就升盘半球超高水。但和欧赔还算比较吻合，看好分胜负，如果考虑到客队伤兵满营的不利情况，可以考虑单3。 02 利沃诺(20) V 01 切　沃(06) VS 巴　里(13) 30切沃最近的表现可以用渐入佳境来形容，先是连败两场，然后是五场不败，球队固定的主力阵型和清晰的打法套路显示出了教练的能力。巴里的表现其实也不差，联赛头轮打平国米，让人惊叹，接着大胜亚特兰大更是出乎意料。虽然说最近只获得了4平1负的成绩，但是其中也逼平了AC米兰和巴勒莫，还是比较有实力的。赔率方面，立博和威廉都支持分胜负，澳盘的盘口就相当诡异，先是主队让平半低水，然后直接就升盘半球超高水。但和欧赔还算比较吻合，看好分胜负，如果考虑到客队伤兵满营的不利情况，可以考虑单3。 02 利沃诺(20) VS 巴勒莫(10) 0  主对作为一支升班马，实力相当有限，以其3平4负仅进1球的联赛成绩显然无法在意甲生存。巴勒莫是典型的主场龙客场虫，8场客场未取一胜，本场比赛能在客场让出平手/半球也算得到了盘口的支持。本人多次提到，客场让球平手/半球超高水，这样的一个盘口是标准的客胜信号。想想上周的国家队比赛，阿塞拜客场赢列支敦士登，巴拉圭客场赢委内瑞拉，还有昨天晚上国米客场5球大胜热那亚，都是这样的一个澳盘盘口所以本场比赛可以单博0。考虑到最近意甲冷门太频繁，有资金的请补个1。 04 帕尔马(09) VS 锡耶纳(17) 3帕尔马是升班马中实力最强的一支，保级不是他们的目标，以球队的球员能力，重回当年的意甲七强不是梦。本场比赛，双方都有两名主力球员受伤无法上场，在实力受影响的情况下，一支升班马球队能让出半球的盘口应该说是相当受到支持的。威廉赔率下2胜1平，立博赔率下5胜1负，新赛季2胜。单3做胆 06 那不勒(15) VS 博洛尼(16) 31那不勒斯这两个赛季都投入了大笔的资金进行球队改造，但是球队并

   我一直觉得我们的很稳定，我和女朋友都是25岁，我们交往三年零两月年，没结婚也没儿没女儿，我很爱我的家。    自认为我是个重感情，重视家庭责任的男人，交往一年多了这么些年来，可以说，我任劳任怨，在外没黑没白的拼搏，现在什么都有了，该给她们的我都做到了，在外人看来，我们是极其幸福的家庭，同事和朋友都很羡慕···可我没想到我女朋友竟会出轨了，她也和我承认了。为了这个家，她求我原谅她，我非常纠结···我们不是没有感情，也因为我对这个家付出的太多了，真正的要离开，我舍不得，还有我想我们离开以后我需要面对的问题会很多，特别是觉得社会的压力很大，   我一直觉得我们的很稳定，我和女朋友都是25岁，我们交往三年零两月年，没结婚也没儿没女儿，我很爱我的家。    自认为我是个重感情，重视家庭责任的男人，交往一年多了这么些年来，可以说，我任劳任怨，在外没黑没白的拼搏，现在什么都有了，该给她们的我都做到了，在外人看来，我们是极其幸福的家庭，同事和朋友都很羡慕···可我没想到我女朋友竟会出轨了，她也和我承认了。为了这个家，她求我原谅她，我非常纠结···我们不是没有感情，也因为我对这个家付出的太多了，真正的要离开，我舍不得，还有我想我们离开以后我需要面对的问题会很多，特别是觉得社会的压力很大，这件事让我觉得会很没有面子.面对亲戚朋友同事会有压力。这件事从我发现到现在快有二十天了，这些天我时常觉得自己快要崩溃了，我忍受不了，也无法从内心里忘掉，我想原谅她，想要这个家，可我似乎很难承受，每天走进家门的时候，我都觉得像一块巨石压在心头，让我喘不过气来···和她认识交往这三多年以来我为我们两人付出了很多，我也很疼爱我的妻子，每天下班回家几乎都是我准备饭菜，我对她的家人也很好，我的女朋友脾气暴躁，这么些年无论她怎样不讲道理，我都在忍耐！我在外面也一直是个说一不二的人，我有自己的事业，而且可以说我很成功，但在家里，我从来都没和她发过火，最难受的时候我也只是隐忍，这些都没有什么，都是一个男人该做的，但我怎么也没想到她会有第三者，也没想到这种事会发生在我的身上，我无法让自己忘掉，回到家我就会想到她和那个男人缠绵的情景，这甚至让我无法呼吸。我不知道自己该怎么办？这段日子我觉得自己至少老了2岁，在这种情况下，我是否该离开她呢呢？<sep>回答：
你好，我不懂！我没用过打印到PS，因为.bmp文件不大清楚

谁知道花生米怎么淹制 <sep>回答：
一、你这是因在日常和工作中，腰部用力不当、过度用力、姿势或体位的不正确等情况。这些长期反复的外力造成的轻微损伤。日积月累地作用于椎问盘，加重了退变的程度。同时诱发因素是突然的腰部负荷增加，尤其是快速弯腰、侧屈或旋转，是形成纤维环破裂的主要原因。二、你也可自我检测是否有腰椎间盘突出：    ● 突然发病，腰痛伴下肢放射性疼痛。    ● 时好时坏，时轻时重，反复发作。    ● 一般先腰痛，后有腿痛。    ● 做增加腹压的动作时症状加重。 另外，你可到正规医院做个CT检查。三、现介绍一套家庭自我保健操，以供你参考：第一节：俯卧位，平卧在硬板床上，做好预备动作。第二节：俯卧位，用双肘关节撑起坚持3分钟，然后复原修整1分钟，重复 6--8次。第三节：俯卧位，用双手撑起，肘关节伸直，坚持3分钟后复原1分钟，重复6—8次。每次可加用深呼吸法，吸一口气，然后吐气，吐尽为止。此时会感觉腰部下沉，使腰椎尽量恢复到原来的正常生理曲度。第四节：利用家中的熨衣板或木板，再加一条安全带或强力皮带，牢牢束住腰部。俯卧位，用双手撑起，坚持3分钟后复原，反复6—8次。第五节：俯卧，腹下放一个枕头，双手扣紧于背后，将双腿、头部和肩膀尽量提起，坚持一秒钟后，然后放下松弛，重复6～8次。第六节：放松动作。做完上述练习后，做一下屈曲膝关节的动作。以正确坐姿休息片刻，即坐在硬椅子上，臀部紧靠椅背根部，若有条件最好是在椅背中央放置一个圆柱形靠背，肩胛骨尽量紧靠椅背，双腿自然放松。第七节：站立伸展。直立，双脚微微分开，手放腰背部，四指并拢。手指向后，以双手作支柱，尽量将腰以上身躯向后弯，双膝要保持挺直。维持一两秒钟，然后回到开始位置。每次重复练习时，尽量尝试将上半身弯得比前一次更后、更弯，以达到最大可能的伸展度。腰突症引发腰腿痛者还应遵守以下三点：1．每晚睡硬板床。2．仰卧位时，腰下垫只薄软的小枕头。3．当腰腿痛急性发作、疼痛剧烈时，应立即卧床休息。如果不能平卧，可侧卧或俯卧。此时不要做上述保健操。在三四天内，应避免做向前弯曲的动作，如弯腰扫地、拖地、洗头等。


征上联：（下联：天涯踏尽红尘） <sep>回答：
    坚持学练气功，有利于改善五脏之间相互配合的关系。　　第一，有利于改善脾与肺相互合作的关系。前人有“脾为生气之源，肺为主气之枢”的说法。就是说脾主运化

和老公结婚两年了。宝宝一岁快半了。老公对宝宝那是真爱的那种，很亲很疼， 我问他生命中最爱的人是谁，他都说是宝宝。问他第二个，他说是下一个宝宝。其实他是不愿意说是他父母还是我。这些我也不计较。只是我们在深圳工作。他是做项目工程的。我是做业务的。我看多了在外搞的男的，现在对老公很担心会出现这个情况。我们不在同一个公司，平进都住公司，周末才回家聚一下。他也要出差，和很多客户要供应商呀打交道。在这个乱性的社会，我们又不是天天在一起。心里很没安全感。今年春天时我有一段时间没上班，他在惠州上班，我们家在东莞，他只有周末才回来。有一次他回来了我就发现他上有个女的和老公结婚两年了。宝宝一岁快半了。老公对宝宝那是真爱的那种，很亲很疼， 我问他生命中最爱的人是谁，他都说是宝宝。问他第二个，他说是下一个宝宝。其实他是不愿意说是他父母还是我。这些我也不计较。只是我们在深圳工作。他是做项目工程的。我是做业务的。我看多了在外搞的男的，现在对老公很担心会出现这个情况。我们不在同一个公司，平进都住公司，周末才回家聚一下。他也要出差，和很多客户要供应商呀打交道。在这个乱性的社会，我们又不是天天在一起。心里很没安全感。今年春天时我有一段时间没上班，他在惠州上班，我们家在东莞，他只有周末才回来。有一次他回来了我就发现他上有个女的发信息给他口气不是一般朋友。我就假装是他说我晚昨上正忙着哄我宝没时间回信息给她。那女的就发住息来说：你都有宝宝了还说想结婚？我就什么都明白了。我问他了，他不承认，说是一般的网友。第二周他回来那女的竟打电话给他，当时我在，他接了说在家就马上挂了。那天他回到公司喝了酒，打电话给我说了了一切，那女的是江西的一网友，小孩三岁了，婚姻并不如意，可能对他有了好感，但两人并没有什么。他就不上那个QQ号了，我又重新帮他申请了一个。只有我在他好友里。还有几个工作上的同事。但是事后到现在，我心里一直没有安全感。有时候不是我老公坏，是这个社会诱惑太多了。男人经不住诱惑，我是一个很传通的人，如果我老公做了对不起我的事，我一定不可能和他再过下去了。我一个人过一辈子也不可能再原谅他。现在虽然没有，可是心理很担心。平时我们感情也还可以，他也知道疼我。<sep>回答：
有GM帮你忙么，外域的稀有精英就相当于副本的大BOSS，你觉得你能抗几下。而却猎人抓宝宝时护甲是0，反正防御战士是单抗不了。


最近打2V2,

一个成功的男人背后会有一个坚强的女人，而一个成功的女人背后却有一? <sep>回答：
红薯，又称甘薯、番薯、山芋。由于地区不同，人们对它的称呼也不同，山东人称其为地瓜，四川人称其为红苕，北京人称其为白薯，福建人称其为红薯。它的故乡是南美洲，16世纪末传入我国。如今，除青藏高原地区外，我国大江南北皆有其踪迹。红薯品种颇多，形状有纺锤、圆筒、椭圆、球形之分；皮色有白、淡黄、黄、红、紫红之别；肉色有黄、杏黄、紫红诸种。 红薯营养十分丰富，是我国人民喜爱的粮菜兼用的天然滋补食品。红薯中含有多种人体需要的营养物质。每500克红薯约可产热能635千卡，含蛋白质11.5克、糖14.5克、脂肪1克、磷100毫克、钙90毫克、铁2克，胡萝卜素0.5毫克，另含有维生素B1、B2、C与尼克酸、亚油酸等。其中维生素B1、B2的含量分别比大米高6倍和3倍。特别是红薯含有丰富的赖氨酸，而大米、面粉恰恰缺乏赖氨酸。此外，红薯与米面混吃，还可发挥蛋白质的互补作用，提高其营养价值。 更有趣的是，清朝乾隆年间，某天，乾隆皇帝在御厨房闻到一股特殊的香味。他好奇地循香寻去，原来是一个小太监在吃烤红薯。乾隆皇帝要了一块，竟吃出滋味来，下令让御厨以后经常为他烤红薯吃。乾隆皇帝吃了一段时间的红薯后，惊奇地发现，他那让太医束手无策的老年性便秘，竟不治而愈了，不仅如此，还胃口大开了。 红薯不仅是甜香益寿食品，还是一种祛病的良药。《本草纲目拾遗》说，红薯能补中、和血、暖胃、肥五脏。《金薯传习录》说它有6种药用价值：治痢疾和泻泄；治酒积和热泻；治湿热和黄疸；治遗精和白浊；治血虚和月经失调；治小儿疳积。《陆川本草》说，红薯能生津止渴，治热病口渴。 红薯含有大量不易被消化酶破坏的纤维素和果胶，能刺激消化液分泌及肠胃蠕动，从而起到通便作用。李时珍的《本草纲目》记载，红薯有“补虚乏，益气力，健脾胃，强肾阴”的功效。“红薯蒸、切、晒、收，充作粮食，称做薯粮，使人长寿少疾。”又有医书说红薯“功同山药，久食益人，为长寿之食”。民间常将红薯与粳米煮稀粥加白糖，用来治疗维生素A缺乏所致的夜盲症、大便带血、便秘、湿热、黄疸、血虚、月经不调等病症。 近年，日本人掀起了一阵又一阵的“红薯热”，男女老少都以大吃特吃形形色色的红薯食品(如烤红薯、红薯酱、红薯酒等)为乐。其原因是，日本家已发现，红薯具有多种特殊的“药用价值”。更令人感兴趣的是：美国

相册走形。。。   只要我把照片按比例缩小到600KB以下照片就走形了怎么办？  <sep>回答：
最理想的彩妆是没有妆的痕迹在脸上，但却明亮动人，呈现出或温柔，或浪或狂野的鲜明风格。新嫁娘多半期望拥有这一生最无暇的美丽。      　1、彩妆开始前先确立脸型修饰及眉型： 　　 将眉型以外的杂毛，如两眉之间、眉峰下方，及眉峰过厚部份拔除。 　　眉型的标准位置-眉头应与眼头齐，超出部份应该去，眉峰应位于黑眼球直视时的外侧正上方，眉头至眉峰应齐宽，东方女子多眉头稀、眉峰厚，应将眉头加深、加厚，眉峰则修薄直到等宽，眉的长度为自鼻翼与眼眉的直线延长，宽度则自眉峰开始逐渐变窄到眉尾。 　　定眉型后，再观察唇部上方的汗毛是否太粗、颜色太深，许多汗毛较粗、颜色较深的女性在此部位即有此问题，如果不处理，不管如何上妆，总令人觉得太男性化，最好定期以刮刀剔去，也不须担心剔去后是否会变得更粗、更黑。 　　2、第二步骤是决定粉底的颜色： 　　 先决定为有光泽或无光泽，油性肌肤应使用无光泽粉底，因油性肌肤已极易出油造成脱妆；干性肌肤应使用光泽粉底，但拍婚纱照时则须使用无光泽粉底以免拍照时反光。 　　 挑选两、三种接近自己肤色的粉底，试擦于脸颊，以确定与自己的肤色最接近颜色，如已有上妆，最好于耳前方局部位置卸妆再试，颜色才会准确。 　　 也可以尝试比自己肤色较深的一号的粉底，如此妆看来为更有立体感的效果。 　　3、如何以粉底修饰脸型是一个很重要的技巧： 　　 运用明、暗色呈现宽、窄效果的方法来修饰肤色，如过宽的两颊或颧骨都可以深肤色粉底来修饰。较窄的额头、扁平的颧骨或过短的下巴，都可以用白色或亮色的粉底来修饰，但与原粉底的接缝处要均匀、柔和，不要明显看出有2色的的差异，为求均匀、柔和的粉底效果，最好以柔软海绵来擦匀粉底，效果更佳。 　　4、上粉： 　　 在家上妆可以蜜粉上妆。如须较浓的效果，可以蜜粉均匀的揉擦再轻按于脸上，如此蜜粉及上粉的效果较持久且不易脱落。淡妆则以粉刷沾粉再自脸部上方至下方、油中到外的顺序刷上，最好使用有附过滤蜜网的蜜粉，可避免沾上过多的粉，使得脸上粉妆浓、淡、厚、薄不均匀。 　　 粉的颜色也须选择最接近肤色，标准的粉底，粉色上完妆后，脸部的颜色应与颈部的肤色相同，才是最自然、正确的色调。如希望有较白肤效果，可以使用较肤色淡一号的粉底及蜜粉。但须在颈部及上衣领口露出部位都上

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>                                                                                │
│                                                                                                  │
│   1 max_length = 512                                                                             │
│   2 for prompt, target in zip(prompts_processed, labels):                                        │
│   3 │   t1 = tokenizer.tokenize(prompt)                                                          │
│ ❱ 4 │   t2 = tokenizer.tokenize(target)                                                          │
│   5 │   if len(t1) + len(t2) > max_length:                                                       │
│   6 │   │   print(f"{prompt}\n{target}\n\n")                                                     │
│   7                                                                                              │
│                                                                                                  │
│ C:\Users\sunzeyeah\AppData\Roaming\Python\Python39\site-packages\transformers\tokenization_utils │
│ .py:547 in tokenize                                                                              │
│                                                                                                  │
│   544 │   │   │   if token in no_split_token:                                                    │
│   545 │   │   │   │   tokenized_text.append(token)                                               │
│   546 │   │   │   else:                                                                          │
│ ❱ 547 │   │   │   │   tokenized_text.extend(self._tokenize(token))                               │
│   548 │   │   # ["This", " is", " something", "<special_token_1>", "else"]                       │
│   549 │   │   return tokenized_text                                                              │
│   550                                                                                            │
│                                                                                                  │
│ C:\Users\sunzeyeah/.cache\huggingface\modules\transformers_modules\local\tokenization_glm.py:564 │
│ in _tokenize                                                                                     │
│                                                                                                  │
│   561 │   │   │   │   encoded_inputs[self.model_input_names[3]] = required_target + [self.igno   │
│   562 │   │   │   elif self.padding_side == "left":                                              │
│   563 │   │   │   │   if return_attention_mask:                                                  │
│ ❱ 564 │   │   │   │   │   # encoded_inputs["attention_mask"] = [0] * difference + encoded_inpu   │
│   565 │   │   │   │   │   attention_mask = encoded_inputs["attention_mask"][0]                   │
│   566 │   │   │   │   │   attention_mask = np.concatenate((np.zeros((len(required_input), diff   │
│   567 │   │   │   │   │   # pad_attention_mask = attention_mask.new_zeros((max_length, differe   │
│                                                                                                  │
│ C:\Users\sunzeyeah\AppData\Roaming\Python\Python39\site-packages\sentencepiece\__init__.py:531   │
│ in Encode                                                                                        │
│                                                                                                  │
│    528 │   │   return self._EncodeAsIds(input, enable_sampling, nbest_size,                      │
│    529 │   │   │   │   │   │   │   │    alpha, add_bos, add_eos, reverse, emit_unk_piece)        │
│    530 │     if out_type is str:                                                                 │
│ ❱  531 │   │   return self._EncodeAsPieces(input, enable_sa

In [9]:
prompt

'今天晚上我在睡觉.........他想要做那些事..我就大大声骂他"不要吵我睡觉"!!!!!...他就跑出去了...还不接我电话<sep>回答：'

In [10]:
target

'你机器里同时有国服和台服是不行的.那样会有冲突的.无法正常运行的.以下办法用下吧.台服客户端下载：  (1)打开CWOW文件夹，找到realmlist.wtf文件，用记事本打开它，先清空里面的内容，再添加“set realmlist  ”（不包括双引号），保存后退出，这样，在每次要进去台服的时候，只需要用WoW.exe来启动，如果是进去CWOW的话，就用Launcher.exe来启动，不会造成冲突。另外，用WoW.exe启动后，登陆的界面是CWOW的界面，这是因为使用的是CWOW的客户端的原因，不用怀疑，直接输入ID密码登陆后进入的就是台服。 ---------------------- (2)如果你已经有CWOW的客户端了，再想装TWOW的时候需要修改注册表。 修改方法： 1、\u3000点击\u3000开始 2、\u3000点击\u3000运行 3、\u3000在运行框里输入\u3000regedit4、\u3000在弹出的注册表框里点击\u3000编辑 5、\u3000点击查找 6、\u3000在查找框里输入\u3000Blizzard Entertainment 7、\u3000当左边框里搜索到\u3000Blizzard Entertainment\u3000的时候，在左边整个删除，一直搜索到出现对话框里说\u3000注册表搜索完毕\u3000为止 [安装] 然后是安装，直接解压，安装就OK了，安装后貌似是1.11.0的版本(视你下载的版本为定)。 --------------------- 补丁方面,直接通过程序下载,速度很快,一般能达到80K/S以上的速度(1M ADSL).---------------------------【注册账号】此处资料为转载，仅供参考------  直接申请体验CDK,然后开始录入个人资料. 召集令编号: RF070112000032 RF070112000033 RF070112000034 RF070112000035 RF070112000036 注：注册帐号时填写"召集令编号"可获得（14天）30小时体验优惠。 非必选项 录入个人资料这里,请注意一下,请尽量填写个人准确资料,如地区选(其它),以及真实身份证号码,真实住址,还有真实的email地址.(^^我选的住在钓鱼岛...不过地址栏下面的都写的真实的) 范例：

In [ ]:
prompt = "问题:移花接玉(密封)能换到什么样法师技能书.或能换多少钱或是什么装配 问题描述:我打到这本的价钱是我少.能换飞混加多少点的或是参换什么技能.要是钱又是能换多少.我是每一次打到高级书的.不清楚这书怎样价钱.会的回答我.我是粤秀区的回答:[MASK]"
target = "6000W-7000W,价格应该和怒神差不多,比魄冰高一些.移花接玉这技能不是很有用,三级之后,给每个宝宝加13点破坏,还得一个一个地加,非常麻烦.砍战士的威力不如砍法师大.(我曾经轮流砍过我的法师小号和战士小号,砍法师还偶尔出绿字,砍战士通常40-50,估计跟战士的物理防御高有关.)打新地图的作用也不大,刚使完移花接玉,抛石兵两个爆裂,宝宝全报销了.也就是挂机还有些用处,再就是和法师PK,配合云寂术效果更佳."
# prompt += "[MASK]"
inputs = tokenizer(prompt, return_tensors="pt")
print(tokenizer.batch_decode(inputs['input_ids']))
print(f"inputs: input ids: {inputs['input_ids'].shape}, attention mask: {inputs['attention_mask'].shape}, token type ids: {inputs['token_type_ids'].shape}\n")
# inputs_glm = tokenizer.build_inputs_for_generation(inputs, max_gen_length=100)
inputs_glm = tokenizer.build_inputs_for_generation(inputs, targets=[target], 
                                                   max_gen_length=200)

# print(inputs_glm)
print(tokenizer.batch_decode(inputs_glm['input_ids']))
print(tokenizer.batch_decode(inputs_glm['labels'][:, 80:]))
print(f"inputs_glm: input ids: {inputs_glm['input_ids'].shape}, attention mask: {inputs_glm['attention_mask'].shape}, token type ids: {inputs_glm['position_ids'].shape}, labels: {inputs_glm['labels'].shape}")
# print(f"inputs_glm: input ids: {inputs_glm['input_ids'].shape}, attention mask: {inputs_glm['generation_attention_mask'].shape}, token type ids: {inputs_glm['position_ids'].shape}")


In [28]:
input_length, target_length = 30, 40
attention_mask = torch.tensor([1]*input_length, dtype=torch.long)
print(attention_mask.shape)
# attention_mask = attention_mask.unsqueeze(1).expand(-1, max_length, -1)
attention_mask = attention_mask.unsqueeze(0).expand(max_length, -1)
print(attention_mask.shape)
t1 = attention_mask.new_zeros((input_length, target_length))
t2 = torch.tril(attention_mask.new_ones((target_length, target_length)))
t3 = attention_mask.new_zeros((max_length-input_length-target_length, target_length))
print(f"t1: {t1.shape}, t2: {t2.shape}")
generation_attention_mask = torch.cat([t1, t2, t3],dim=0)#.unsqueeze(0).expand(-1, -1)
print(generation_attention_mask.shape)
pad_attention_mask = attention_mask.new_zeros((max_length, max_length-input_length-target_length))
print(pad_attention_mask.shape)
attention_mask = torch.cat((attention_mask, generation_attention_mask, pad_attention_mask), dim=1)
# attention_mask = attention_mask.unsqueeze(1)
print(attention_mask.shape)

torch.Size([30])
torch.Size([512, 30])
t1: torch.Size([30, 40]), t2: torch.Size([40, 40])
torch.Size([512, 40])
torch.Size([512, 442])
torch.Size([512, 512])


# SFT Prediction

In [ ]:
# prefix = "模型回答："
prefix = "答:"

In [ ]:
if "glm" in model_name_or_path:
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path, trust_remote_code=True)
else:
    model = AutoModelForCausalLM.from_pretrained(model_name_or_path, trust_remote_code=True, use_cache=False)
    model.resize_token_embeddings(tokenizer.vocab_size)
model.to(device)
print(model.device)

In [ ]:
checkpoint_files = os.path.join(root, "chatgpt", "output", "sft", "pangu-350M", "external_checkpoint-12000", "pytorch_model*.bin")
# checkpoint_files = os.path.join(root, "chatgpt", "output", "sft", "pangu-2.6B", "external_checkpoint-9000", "pytorch_model*.bin")
checkpoints = glob.glob(checkpoint_files)
st = dict()
for checkpoint in checkpoints:
    st.update(torch.load(checkpoint, map_location="cpu"))
model.load_state_dict(st)

In [ ]:
# text_generator = TextGenerationPipeline(model, tokenizer, device=device)

In [ ]:
max_length = 512
max_length_generation = 50
num_return_sequences = 1
top_p = 0.8
temperature = 0.8
prompt = "写一篇歌颂祖国的文章"
# prompt_processed = prompt + tokenizer.sep_token + prefix
# prompt = """阅读文章：《战国无双3》（）是由光荣和ω-force开发的战国无双系列的正统第三续作。本作以三大故事为主轴，分别是以武田信玄等人为主的《关东三国志》，织田信长等人为主的《战国三杰》，石田三成等人为主的《关原的年轻武者》，丰富游戏内的剧情。此部份专门介绍角色，欲知武器情报、奥义字或擅长攻击类型等，请至战国无双系列1.由于乡里大辅先生因故去世，不得不寻找其他声优接手。从猛将传 and Z开始。2.战国无双 编年史的原创男女主角亦有专属声优。此模式是任天堂游戏谜之村雨城改编的新增模式。本作中共有20张战场地图（不含村雨城），后来发行的猛将传再新增3张战场地图。但游戏内战役数量繁多，部分地图会有兼用的状况，战役虚实则是以光荣发行的2本「战国无双3 人物真书」内容为主，以下是相关介绍。（注：前方加☆者为猛将传新增关卡及地图。）合并本篇和猛将传的内容，村雨城模式剔除，战国史模式可直接游玩。主打两大模式「战史演武」&「争霸演武」。系列作品外传作品\n问：《战国无双3》是由哪两个公司合作开发的？\n答："""
while True:
    if "glm" in model_name_or_path:
#         prompt += tokenizer.mask_token
        prompt += "[gMASK]"
        inputs = tokenizer(prompt, return_tensors="pt")
        inputs = tokenizer.build_inputs_for_generation(inputs, max_gen_length=max_length + max_length_generation)
        inputs = inputs.to(device)
        outputs = model.generate(**inputs,
                                 max_new_tokens=max_length_generation,
                                 eos_token_id=tokenizer.eop_token_id,
                                 pad_token_id=tokenizer.pad_token_id,
                                 do_sample=False,
                                 num_return_sequences=num_return_sequences,
                                 top_p=top_p,
                                 temperature=temperature)
        
    else:
        inputs = tokenizer(prompt, add_special_tokens=False, return_token_type_ids=False, return_tensors="pt")
        inputs = inputs.to(device)
        outputs = model.generate(**inputs,
                                 max_new_tokens=max_length_generation,
                                 pad_token_id=tokenizer.pad_token_id,
                                 do_sample=False,
                                 num_return_sequences=num_return_sequences,
                                 top_p=top_p,
                                 temperature=temperature)
        # outputs = text_generator(prompt, max_length=args.max_length_generation,
        #                          do_sample=True, num_return_sequences=args.num_return_sequences,
        #                          top_p=args.top_p, temperature=args.temperature)
#         results = [output['generated_text'].split(prefix, maxsplit=1)[1].replace(tokenizer.eos_token, "").replace(tokenizer.pad_token, "") for output in outputs]
    results = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    results = [result.split(prefix, maxsplit=1)[1] for result in results]
    break

In [ ]:
print(tokenizer.batch_decode(inputs['input_ids']))
print(tokenizer.batch_decode(outputs))

In [ ]:
outputs

In [5]:
# f = os.path.join(root, "raw", "baike_qa_train.json")
# [baike_qa.jsonl, chinese_classical.jsonl, chinese_poetry.jsonl, couplets.jsonl, weibo_summary_comments.jsonl, zhidao.jsonl]
f = os.path.join(root, "chatgpt", "processed", "baike_qa.jsonl")
i = 0
prompts = []
prompts_processed = []
labels = []
with open(f, "r", encoding="utf-8") as r:
    while True:
        line = r.readline()
        if not line:
            break
        item = json.loads(line.strip("\n"))
        # prompt = clean_text(item['title'] if len(item['title']) > len(item['desc']) else item['desc'])
        # prompt_processed = prompt + tokenizer.sep_token + prefix
        # label = clean_text(item['answer'])
        prompt = item['prompt']
        prompt_processed = prompt
        label = item['answers'][0]['answer']
        prompts.append(prompt)
        prompts_processed.append(prompt_processed)
        labels.append(label)
        i += 1
        # if i > 1000:
        #     break
random.shuffle(prompts_processed)
print(len(prompts_processed))

1470142


In [ ]:
i = 79
num_return_sequences = 2
max_length = 512
max_length_generation = 100
top_k = 50
top_p = 0.8
temperature = 1.0
t1 = time.time()
prompt = prompts_processed[i]
inputs = tokenizer(prompt, add_special_tokens=False, return_token_type_ids=False, return_tensors="pt")
inputs = inputs.to(device)
outputs = model.generate(**inputs,
                         max_new_tokens=max_length_generation,
                         pad_token_id=tokenizer.pad_token_id,
                         do_sample=True,
                         num_return_sequences=num_return_sequences,
                         # top_p=top_p,
                         top_k=top_k,
                         temperature=temperature)
results = tokenizer.batch_decode(outputs, skip_special_tokens=True)
for result in results:
    # result.split(prefix, maxsplit=1)[1]
    print(result)


# results = text_generator(prompts_processed[i:j], max_length=200, num_return_sequences=num_return_sequences,
#                          do_sample=True, top_k=50, temperature=10.0)
# print(f"Finished prediction, time taken: {time.time()-t1}")

# for prompt, res, label in zip(prompts[i:j], results[:(j-i)], labels[i:j]):
#     print(f"prompt: {prompt}\nlabel: {label}")
#     for k in range(num_return_sequences):
#         model_answer = res[k]['generated_text'].split(prefix)[1].replace("<eot>", "").replace("<pad>", "")
#         print(f"model answer-{k}: {model_answer}")
#     print("\n\n")

# Reward Model

In [ ]:
model_name_or_path = "D:\\Data\\models\\pangu-350M"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_cache=False, trust_remote_code=True)
tokenizer.add_special_tokens({'unk_token': "<unk>",
                                  'bos_token': "<s>",
                                  'eos_token': "<eot>",
                                  'pad_token': "<pad>",
                                  "sep_token": "<sep>"})

In [ ]:
max_length = 1024
text = "你好，你是谁"
# text = "<|startoftext|>" + text + "<|endoftext|>"
res = tokenizer(text, max_length=max_length, truncation="longest_first", 
          return_tensors="pt", add_special_tokens=False)

In [ ]:
res.keys()
# torch.cat((res['input_ids'], res['input_ids']), axis=1)

# Split torch checkpoint into multiple checkpoints

In [ ]:
checkpoint = os.path.join(root, "chatgpt", "output", "sft", "pangu-2.6B", "checkpoint-42782")
if "glm" in model_name_or_path:
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, trust_remote_code=True)
else:
    model = AutoModelForCausalLM.from_pretrained(checkpoint, trust_remote_code=True, use_cache=False)
st = model.state_dict()

In [ ]:
import sys
print(sys.getsizeof(st))
keys = list(st.keys())
n = 10
m = {"metadata": {"total_size":sys.getsizeof(st)}, "weight_map":dict()}
span = len(keys) // n
for i in range(n):
    fn = f"pytorch_model-{i+1}-of-{n}.bin"
    f = os.path.join(checkpoint, fn)
    stt = dict()
    for key in keys[i*span:(i+1)*span]:
        stt[key] = st[key]
        m["weight_map"][key] = fn
    torch.save(stt, f)
f = os.path.join(checkpoint, "pytorch_model.bin.index.json")
json.dump(m, open(f, "w", encoding="utf-8"), ensure_ascii=False)

# Data Processing

### weibo_summary_comments_json

In [ ]:
t = time.time()
fi = os.path.join(root, "raw", "weibo_summary_comments_json.json")
fo = os.path.join(root, "chatgpt", "processed", "weibo_summary_comments.jsonl")
ct = 0
with open(fo, "w", encoding="utf-8") as w:
    with open(fi, "r", encoding="utf-8") as r:
        while True:
            line = r.readline()
            if not line
                break
            
            item = json.loads(line.strip("\n"))
            article = item['article'].replace(" ", "")
            abstract = item['abstract'].replace(" ", "")
            prompt = f"新闻内容：{article}{tokenizer.sep_token}摘要：{abstract}{tokenizer.sep_token}评论："
            answers = [
                {
                    "answer": k.replace(" ", ""), 
                    "score": int(v)
                } for (k, v) in sorted(item['comments'], key=lambda x: (int(x[1]), len(x[0])), reverse=True)
            ]
            w.write(json.dumps({"prompt": prompt, "answers": answers}, ensure_ascii=False)+'\n')
            ct += 1
print(f"length: {ct}, time taken: {time.time()-t} s")

### couplets

In [ ]:
t1 = time.time()
fi = os.path.join(root, "raw", "couplets.txt")
fo = os.path.join(root, "chatgpt", "processed", "couplets.jsonl")
l2 = []
nexts = dict()
with open(fi, "r", encoding="utf-8") as r:
    while True:
        line = r.readline()
        if not line:
            break
        line = line.strip("\n")
        idx = len(line) // 2
        prompt = line[:idx]
        answer = line[idx+1:]
        answers = [{"answer": answer, "score": 1}]
        l2.append({"prompt": f"上联：{prompt}{tokenizer.sep_token}下联：", "answers": answers})
        length = len(answer)
        if length not in nexts:
            nexts[length] = list()
        nexts[length].append(answer)
t2 = time.time()
print(f"length: {len(l2)}, # different lengths: {len(nexts)}, time taken: {t2-t1} s")
with open(fo, "w", encoding="utf-8") as w:
    for i, l in tqdm(enumerate(l2), desc="Processing Couplets"):
        answer = l['answers'][0]
        length = len(answer['answer'])
        # 上下联长度一样
        nexts_tmp = set(nexts[length])
        nexts_tmp.remove(answer['answer'])
        nexts_tmp = set(nexts[length]).difference(set([answer['answer']]))
#         nexts_tmp.remove(answer['answer'])
        answers.extend([{"answer": fa, "score": 0} for fa in random.sample(nexts_tmp, 2)])
        # 上下联长度不一样
        keys = set(nexts.keys())
        keys.remove(length)
        answers.extend([{"answer": random.choice(nexts[key]), "score": -1} for key in random.sample(keys, 2)])
#         answers = sorted(answers, key=lambda x: x['score'], reverse=True)
        w.write(json.dumps({"prompt": l['prompt'], "answers": answers}, ensure_ascii=False)+'\n')
#         if i % 1000 == 0:
#             print(f"{i} samples processed, time taken: {time.time()-t2} s")
print(f"length: {len(l2)}, time taken: {time.time()-t2} s")

### zhidao

In [ ]:
t = time.time()
fp = os.path.join(root, "raw", "zhidao", "*.csv")
fo = os.path.join(root, "chatgpt", "processed", "zhidao.jsonl")
ct = 0
with open(fo, "w", encoding="utf-8") as w:
    for fi in glob.glob(fp):
        ct = 0
        df = pd.read_csv(fi).sort_values(by=["title", "is_best"], ascending=False)
        prev_title = None
        prev_prompt = None
        for _, val in df.iterrows():
            if isinstance(val['question'], str) and val['question'] != val['title']:
                prompt = f"问题：{val['title']}{tokenizer.sep_token}内容：{val['question']}{tokenizer.sep_token}回答："
            else:
                prompt = f"问题：{val['title']}{tokenizer.sep_token}回答："
            if prev_title is not None and prev_title == val['title']:
                answers.append({"answer": val['reply'], "score": val['is_best']})
            else:
                if prev_title is not None:
#                     l3.append({"prompt": prev_prompt, "answers": copy.deepcopy(answers)})
                    w.write(json.dumps({"prompt": prev_prompt, "answers": answers}, ensure_ascii=False)+'\n')
                answers = [{"answer": val['reply'], "score": val['is_best']}]
            prev_prompt = prompt
            prev_title = val['title']
            ct += 1
#         l3.append({"prompt": prev_prompt, "answers": copy.deepcopy(answers)})
        w.write(json.dumps({"prompt": prev_prompt, "answers": answers}, ensure_ascii=False)+'\n')
        print(f"finished processing {os.path.basename(fi)}")
print(f"length: {ct}, time taken: {time.time()-t} s")

### JDData

In [ ]:
from html.parser import HTMLParser
class MyHTMLParser(HTMLParser):
    def __init__(self):
        super().__init__()
       #Initializing lists
        self.start_tags = list()
        self.end_tags = list()
        self.start_end_tags = list()
        self.data_list = list()
    #HTML Parser Methods
    def handle_starttag(self, startTag, attrs):
        self.start_tags.append(startTag)
    def handle_endtag(self, endTag):
        self.end_tags.append(endTag)
    def handle_startendtag(self,startendTag, attrs):
        self.start_end_tags.append(startendTag)
    def handle_data(self, data):
        self.data_list.append(data)
        
t = time.time()
fi = os.path.join(root, "raw", "JDData", "*.data*")
# fo = os.path.join(root, "chatgpt", "processed", "zhidao.jsonl")
ct = 0
with open(fo, "w", encoding="utf-8") as w:
    for fi in glob.glob(fp):
        ct = 0
        with open(fi, "r", encoding="gbk") as r:
            line = r.readline()
            items = line.strip("\n").split("\t")
            parser = MyHTMLParser()
            parser.feed(items[1])
            for t, d in zip(parser.start_tags, parser.data_list):
                print(f"{t}: {d}")
#                 prompt = f"问题：{val['title']}{tokenizer.sep_token}内容：{val['question']}{tokenizer.sep_token}回答："
#                 answers.append({"answer": val['reply'], "score": val['is_best']})
            ct += 1
#         l3.append({"prompt": prev_prompt, "answers": copy.deepcopy(answers)})
#         w.write(json.dumps({"prompt": prev_prompt, "answers": answers}, ensure_ascii=False)+'\n')
        print(f"finished processing {os.path.basename(fi)}")
print(f"length: {ct}, time taken: {time.time()-t} s")

### yf_amazon

In [ ]:
t = time.time()
fi = os.path.join(root, "raw", "yf_amazon", "products.csv")
dfp = pd.read_csv(fi)
fi = os.path.join(root, "raw", "yf_amazon", "ratings.csv")
dfr = pd.read_csv(fi)
fi = os.path.join(root, "raw", "yf_amazon", "categories.csv")
dfc = pd.read_csv(fi)

In [ ]:
dfp.columns
# dfp['name'].unique().tolist()

In [ ]:
dfp['cate_id_1'] = dfp['catIds'].apply(lambda x: x.split(",")[0])
for cid1 in dfp['cate_id_1'].unique():
    print(dfc[dfc['catId']==int(cid1)]['category'])

### dmsc

In [ ]:
t = time.time()
fi = os.path.join(root, "raw", "dmsc", "movies.csv")
dfm = pd.read_csv(fi)
print(dfm.shape)
fi = os.path.join(root, "raw", "dmsc", "ratings.csv")
dfr = pd.read_csv(fi)
print(dfr.shape)

In [ ]:
dfr.groupby("movieId", 'rating').count()['comment']

### Chinese Classical-Modern

In [ ]:
t1 = time.time()
fp = os.path.join(root, "raw", "Classical-Modern", "bitext", "*")
fo = os.path.join(root, "chatgpt", "processed", "chinese_classical.jsonl")
l3 = []
dicts = dict()
for fi in glob.glob(fp):
    name = os.path.basename(fi)
    dicts[name] = {"古文": [], "现代文": []}
    with open(fi, "r", encoding="utf-8") as r:
        for i, line in enumerate(r):
            line = line.strip("\n")
            if line.startswith("古文"):
                p1 = line[3:]
                dicts[name]['古文'].append(p1)
            elif line.startswith("现代文"):
                p2 = line[4:]
                dicts[name]['现代文'].append(p2)
            elif p1 is not None and p2 is not None:
                pair = [("古文", p1), ("现代文", p2)]
                random.shuffle(pair)
                prompt = f"{pair[0][0]}：{pair[0][1]}{tokenizer.sep_token}{pair[1][0]}："
                answers = [{"answer": pair[1][1], "score": 1}]
                l3.append({"prompt": prompt, "answers": answers, "name": name})
                p1 = None
                p2 = None
t2 = time.time()
print(f"length: {len(l3)}, # different names: {len(dicts)}, time taken: {t2-t1} s")
with open(fo, "w", encoding="utf-8") as w:
    for i, l in tqdm(enumerate(l3), desc="Processing Chinese Classical-Modern"):
        name = l['name']
        prompt = l['prompt']
        answer = l['answers'][0]['answer']
        if prompt.startswith("古文"):
            answer_type = '现代文'
        else:
            answer_type = '古文'
        samples_tmp = set(dicts[name][answer_type])
        samples_tmp.remove(answer)
        answers.extend([{"answer": fa, "score": 0} for fa in random.sample(samples_tmp, 2)])
        keys = set(dicts.keys())
        keys.remove(name)
        answers.extend([{"answer": random.choice(dicts[key][answer_type]), "score": -1} for key in random.sample(keys, 2)])
        w.write(json.dumps({"prompt": prompt, "answers": answers}, ensure_ascii=False)+'\n')
#         if i % 100 == 0:
#             print(f"{i} samples processed, time taken: {time.time()-t2} s")
print(f"length: {i}, time taken: {time.time()-t} s")

### Chinese Poetry

In [ ]:
import opencc
converter = opencc.OpenCC('t2s.json')
t1 = time.time()
fp = [
    # 四书五经
    os.path.join(root, "raw", "chinese-poetry", "lunyu", "lunyu.json"),
#     os.path.join(root, "raw", "chinese-poetry", "mengxue", "*.json"),
    os.path.join(root, "raw", "chinese-poetry", "sishuwujing", "*.json"),
    # 古体诗
    os.path.join(root, "raw", "chinese-poetry", "caocaoshiji", "caocao.json"),
    os.path.join(root, "raw", "chinese-poetry", "shijing", "shijing.json"),
    # 楚辞
    os.path.join(root, "raw", "chinese-poetry", "chuci", "chuci.json"),
    # 诗
    os.path.join(root, "raw", "chinese-poetry", "shi", "poet*.json"),
    # 词
    os.path.join(root, "raw", "chinese-poetry", "ci", "ci*.json"),
    os.path.join(root, "raw", "chinese-poetry", "nalanxingde", "*.json"),
    os.path.join(root, "raw", "chinese-poetry", "wudai", "huajianji", "*juan.json"),
    os.path.join(root, "raw", "chinese-poetry", "wudai", "nantang", "poetrys.json"),
    # 曲
    os.path.join(root, "raw", "chinese-poetry", "yuanqu", "yuanqu.json"),
]
fs = [each for f in fp for each in glob.glob(f)]

l5 = []
dicts = dict()
for fi in fs:
    lines = json.load(open(fi, "r", encoding="utf-8"))
    if isinstance(lines, dict):
        lines = [lines]
    for i, line in enumerate(lines):
        if "lunyu" in fi:
            author = "孔子"
            genre = "经书"
            title = line['chapter']
            contents = "".join(line['paragraphs'])
        elif "daxue" in fi:
            author = "曾子"
            genre = "经书"
            title = "大学"
            contents = converter.convert("".join(line['paragraphs'])).replace("「", "“").replace("」", "”")
        elif "mengzi" in fi:
            author = "孟子"
            genre = "经书"
            title = converter.convert(line['chapter'])
            contents = converter.convert("".join(line['paragraphs'])).replace("「", "“").replace("」", "”")
        elif "zhongyong" in fi:
            author = "孔伋"
            genre = "经书"
            title = "中庸"
            contents = converter.convert("".join(line['paragraphs'])).replace("「", "“").replace("」", "”")
        elif "caocao" in fi:
            author = "曹操"
            genre = "古体诗"
            title = line['title']
            contents = "".join(line['paragraphs'])
        elif "shijing" in fi:
            author = "诗经"
            genre = "古体诗"
            title = line['chapter'] + "-" + line['section'] + "-" + line['title']
            contents = "".join(line['content'])
        elif "chuci" in fi:
            author = line['author']
            genre = "楚辞"
            title = line['section'] + "-" + line['title']
            contents = "".join(line['content'])
        elif "nalanxingde" in fi:
            author = line['author']
            genre = "词"
            title = line['title']
            contents = "".join(line['para'])
        elif "huajianci" in fi:
            author = line['author']
            genre = "词"
            title = line['title']
            contents = "".join(line['paragraphs'])
        elif "nantang" in fi:
            author = line['author']
            genre = "词"
            title = line['title']
            contents = "".join(line['paragraphs'])
        elif "yuanqu" in fi:
            author = line['author']
            genre = "曲"
            title = line['title']
            contents = "".join(line['paragraphs'])
        elif "shi" in fi:
            if len(line['paragraphs']) <= 0:
                continue
            author = converter.convert(line['author'])
            genre = "五言诗" if len(line['paragraphs'][0]) == 12 else "七言诗"
            title = converter.convert(line['title'])
            contents = converter.convert("".join(line['paragraphs']))
        elif "ci" in fi:
            author = line['author']
            genre = "词"
            title = line['rhythmic']
            contents = "".join(line['paragraphs'])
        if genre not in dicts:
            dicts[genre] = dict()
        if author not in dicts[genre]:
            dicts[genre][author] = dict()
        quantifier = "篇" if genre in ["经书", "楚辞"] else "首"
        prompt = f"以{author}的风格，写一{quantifier}{genre}，题为{title}{tokenizer.sep_token}"
        answers = [{"answer": contents, "score": 1}]
        l5.append({"prompt": prompt, "answers": answers, "genre": genre, "title": title, "author": author})
        dicts[genre][author][title] = contents
        
t2 = time.time()
print(f"length: {len(l5)}, # different lengths: {len(dicts)}, time taken: {t2-t1} s")
fo = os.path.join(root, "chatgpt", "processed", "chinese_poetry.jsonl")
with open(fo, "w", encoding="utf-8") as w:
    for i, l in tqdm(enumerate(l5), desc="Processing Chinese Poetry"):
        genre = l['genre']
        author = l['author']
        title = l['title']
        prompt = l['prompt']
        answers = l['answers']
        # 同作者其他作品-2
        titles_tmp = set(dicts[genre][author].keys())
        titles_tmp.remove(title)
        if len(titles_tmp) > 0:
            t = random.choice(list(titles_tmp))
            answers.append({"answer": dicts[genre][author][t], "score": 0})
        # 同体裁其他作者其他作品-1
        authors_tmp = set(dicts[genre].keys())
        authors_tmp.remove(author)
        a = random.choice(list(authors_tmp))
        t = random.choice(list(dicts[genre][a].keys()))
        answers.append({"answer": dicts[genre][a][t], "score": -1})
        # 不同体裁作品-0
        genres_tmp = set(dicts.keys())
        genres_tmp.remove(genre)
        g = random.choice(list(genres_tmp))
        a = random.choice(list(dicts[g].keys()))
        t = random.choice(list(dicts[g][a].keys()))
        answers.append({"answer": dicts[g][a][t], "score": -2})
        w.write(json.dumps({"prompt": prompt, "answers": answers}, ensure_ascii=False)+'\n')
print(f"length: {i}, time taken: {time.time()-t2} s")

### baike_qa_2019

In [ ]:
fs = glob.glob(os.path.join(root, "raw", "baike_qa2019", "baike_qa_*.json"))
fo = os.path.join(root, "chatgpt", "processed", "baike_qa.jsonl")
ct = 0
# items = []
# lens_prompt = []
# lens_label = []
with open(fo, "w", encoding="utf-8") as w:
    for f in fs:
        with open(f, "r", encoding="utf-8") as r:
            while True:
                line = r.readline()
                if not line:
                    break
                item = json.loads(line.strip("\n"))
                question = clean_text(item['title'] if len(item['title']) > len(item['desc']) else item['desc'])
                prompt = f"{question}{tokenizer.sep_token}回答："
                answer = clean_text(item['answer'])
                answers = [{"answer": answer, "score": 1}]
#                 items.append(item)
#                 lens_prompt.append(len(prompt))
#                 lens_label.append(len(label))
                w.write(json.dumps({"prompt": prompt, "answers": answers}, ensure_ascii=False)+'\n')
                ct += 1
print(ct)
# print(len(items))
# print(np.percentile(lens_prompt, np.arange(90, 101)))
# print(np.percentile(lens_label, np.arange(90, 101)))

### rm-static

In [ ]:
fi = os.path.join(root, "raw", "rm-static", "data", "test-00000-of-00001-bf4c733542e35fcb.parquet")
df = pd.read_parquet(fi)
print(df.shape)
df.head(2)

# ChatGLM

In [ ]:
model = AutoModel.from_pretrained(model_name_or_path, trust_remote_code=True).half().cuda()

In [ ]:
texts = [
    "你好",
    "晚上睡不着应该怎么办"
]
history = []
for text in texts:
    response, history = model.chat(tokenizer, text, history=history)
    print(f"问: {text}\n答:{response}\n")